In [3]:
import numpy as np
import copy
from collections import deque

I realise, I could just implement a fucking deque to do all this. It'll be way simpler and should handle all the errors I'm getting.

Sigh.

In [27]:
class LSystemBoardGen:
    def __init__(self, size, agent_count) -> None:
        self.size = size
        self.agent_count = agent_count
        self.board = self.initialise_starting_board(self.size, self.agent_count)
        self.agents = self.assign_agent_states()
        self.og_board = copy.deepcopy(self.board)
        self.og_agents = copy.deepcopy(self.agents)

    def initialise_starting_board(self, size:int, agent_count):
        """Initialise as starting board of shape (size, size) with agent_count agents."""
        # :TO UPDATE
        board = np.zeros(shape=(size,size), dtype=int)
        for agent in range(1, 1+agent_count):
            tries = 0
            while(1):
                tries += 1
                if tries == 100:
                    print(f'ERROR - NO BOARD SPACES FOUND AFTER {100} TRIES')
                    break

                idx0 = np.random.randint(0, self.size)
                idx1 = np.random.randint(0, self.size)
                if board[idx0, idx1] != 0.0:
                    continue
                else:
                    board[idx0, idx1] = agent
                    break
        return board
    
    def reset(self):
        """Reset board back to its original state."""
        self.board = self.og_board
        self.agents = self.og_agents

    def assign_agent_states(self):
        """Construct agent from board info. 
        
        Each agent has the following attributes:
        (agent_num, deque)
        """
        agents = []
        for agent_num in range(1, 1+self.agent_count):
            pos = np.argwhere(self.board==agent_num)
            agent_deque = deque(pos)
            agents.append([agent_num, agent_deque])
        return agents

    def find_next_pos(self, loc:np.ndarray):
        """Find the next empty spot for an agent head/tail."""
        empty_places = np.argwhere(self.board==0)
        empty_place_distances = np.sqrt(np.sum((loc-empty_places)**2, axis=1))
        empty_neighbour_places = np.argwhere(empty_place_distances<1+1/9)
        empty_neighbour_coords = np.take(empty_places, empty_neighbour_places, axis=0)
        return empty_neighbour_coords[:,0,:]

    def push(self, agent_num):
        agent = self.agents[agent_num-1]
        growth_choice = np.random.randint(-1, 1) # choose to grow from the head or the tail, :TO UPDATE
        nbs = self.find_next_pos(agent[1][growth_choice])

        if len(nbs) == 0:
            return None
        
        else:
            growth_loc = np.random.randint(0, len(nbs)) # choose a random growth direction
            
            # grow into the chosen location
            self.board[tuple(nbs[growth_loc])] = agent[0]
            
            # update deque to account for new growth
            if growth_choice == 0:
                agent[1].appendleft(nbs[growth_loc])
            elif growth_choice == -1:
                agent[1].append(nbs[growth_loc])

    def pull(self, agent_num):
        agent = self.agents[agent_num-1]
        shrink_choice = np.random.randint(-1, 1) #:TO UPDATE

        if len(agent[1]) == 1:
            return None
        
        else:
            self.board[tuple(agent[1][shrink_choice])] = 0
            if shrink_choice == 0:
                _ = agent[1].popleft()
            elif shrink_choice == -1:
                _ = agent[1].pop()

In [30]:
Board = LSystemBoardGen(5, 3)
print(Board.board)


Board.push(3)
print(Board.agents[2])
print(Board.board)

Board.push(3)
print(Board.agents[2])
print(Board.board)

Board.push(3)
print(Board.agents[2])
print(Board.board)

Board.pull(3)
print(Board.agents[2])
print(Board.board)

Board.pull(3)
print(Board.agents[2])
print(Board.board)

[[0 0 0 1 0]
 [3 0 0 2 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[3, deque([array([2, 0]), array([1, 0])])]
[[0 0 0 1 0]
 [3 0 0 2 0]
 [3 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[3, deque([array([2, 1]), array([2, 0]), array([1, 0])])]
[[0 0 0 1 0]
 [3 0 0 2 0]
 [3 3 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[3, deque([array([1, 1]), array([2, 1]), array([2, 0]), array([1, 0])])]
[[0 0 0 1 0]
 [3 3 0 2 0]
 [3 3 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[3, deque([array([2, 1]), array([2, 0]), array([1, 0])])]
[[0 0 0 1 0]
 [3 0 0 2 0]
 [3 3 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[3, deque([array([2, 0]), array([1, 0])])]
[[0 0 0 1 0]
 [3 0 0 2 0]
 [3 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
